##**ML Project - Credit Card Fraud Detection**

In [189]:
# First, we'll import Pandas, Numpy a data processing and CSV file I/O library
import pandas as pd
import numpy as np

# We'll also import seaborn, a Python graphing library
import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white", color_codes=True)

In [190]:
# Next, we'll load the dataset, which is in the "../input/" directory
df = pd.read_csv('/content/creditcard.csv') # the dataset is now a Pandas DataFrame

# Let's see what's in the data - Jupyter notebooks print the result of the last thing you do
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [191]:
credit_card_data.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
279186,168697.0,-1.448540,-0.106524,1.590888,0.787416,0.843798,0.719463,0.358887,0.409272,-0.557388,-0.237061,0.424899,0.512476,0.250934,0.135446,0.258259,0.154100,-0.618690,0.720465,1.060753,0.767129,-0.082357,-0.622610,0.248731,0.137849,0.462997,-0.684516,0.271892,0.166265,157.00,0.0
279187,168698.0,-1.698935,1.318773,-0.532258,0.115716,-2.031122,-0.712686,-1.078152,1.537223,0.065154,-1.085665,-1.350958,0.670474,0.047616,0.922667,0.030475,0.426921,0.442459,0.128094,-0.241123,-0.706093,0.524217,1.101406,0.115851,0.473729,-0.908005,0.506611,-0.695470,-0.224998,74.90,0.0
279188,168698.0,-1.698935,1.318773,-0.532258,0.115716,-2.031122,-0.712686,-1.078152,1.537223,0.065154,-1.085665,-1.350958,0.670474,0.047616,0.922667,0.030475,0.426921,0.442459,0.128094,-0.241123,-0.706093,0.524217,1.101406,0.115851,0.473729,-0.908005,0.506611,-0.695470,-0.224998,74.90,0.0
279189,168698.0,-1.752086,1.235061,-0.495090,0.136284,-1.842112,-0.841210,-1.254498,1.572910,0.075590,-1.064181,-1.357656,0.688060,0.046872,0.935071,0.028346,0.412006,0.451100,0.131452,-0.244710,-0.637330,0.549229,1.086182,0.100710,0.471034,-0.914791,0.501103,-0.744175,-0.204416,32.48,0.0
279190,168698.0,-1.752086,1.235061,-0.495090,0.136284,-1.842112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Perform data cleanup as required

In [192]:
credit_card_data.shape

(279191, 31)

In [193]:
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279191 entries, 0 to 279190
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    279191 non-null  float64
 1   V1      279191 non-null  float64
 2   V2      279191 non-null  float64
 3   V3      279191 non-null  float64
 4   V4      279191 non-null  float64
 5   V5      279191 non-null  float64
 6   V6      279190 non-null  float64
 7   V7      279190 non-null  float64
 8   V8      279190 non-null  float64
 9   V9      279190 non-null  float64
 10  V10     279190 non-null  float64
 11  V11     279190 non-null  float64
 12  V12     279190 non-null  float64
 13  V13     279190 non-null  float64
 14  V14     279190 non-null  float64
 15  V15     279190 non-null  float64
 16  V16     279190 non-null  float64
 17  V17     279190 non-null  float64
 18  V18     279190 non-null  float64
 19  V19     279190 non-null  float64
 20  V20     279190 non-null  float64
 21  V21     27

In [194]:
# distribution of legit transactions & fraudulent transactions
credit_card_data['Class'].value_counts()

0.0    278703
1.0       487
Name: Class, dtype: int64

This Dataset is highly unblanced

0 --> Normal Transaction

1 --> fraudulent transaction

In [195]:
# separating the data for analysis
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]

In [196]:
print(legit.shape)
print(fraud.shape)

(278703, 31)
(487, 31)


In [197]:
# statistical measures of the data
legit.Amount.describe()

count    278703.000000
mean         88.676558
std         250.460854
min           0.000000
25%           5.760000
50%          22.190000
75%          77.850000
max       25691.160000
Name: Amount, dtype: float64

In [198]:
fraud.Amount.describe()

count     487.000000
mean      121.913326
std       257.561705
min         0.000000
25%         1.000000
50%         9.130000
75%       105.350000
max      2125.870000
Name: Amount, dtype: float64

In [199]:
# compare the values for both transactions
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,93312.148890,0.007147,-0.013057,0.028317,-0.005329,-0.000463,0.005110,0.007360,-0.001612,0.004745,0.010765,0.000264,0.005956,0.000720,0.013838,0.004085,0.006892,0.012391,0.001862,-0.001105,-0.000309,-0.001446,-0.001745,-0.000874,-0.000160,0.002740,-0.000045,-0.000292,-0.000088,88.676558
1.0,79834.240246,-4.816122,3.652183,-7.065000,4.576639,-3.179607,-1.398985,-5.612526,0.570034,-2.597563,-5.697347,3.820310,-6.291644,-0.103899,-6.993408,-0.090565,-4.164614,-6.700220,-2.263639,0.683637,0.372003,0.716151,0.014237,-0.041043,-0.104202,0.041249,0.047424,0.168312,0.075910,121.913326


Under-Sampling

Build a sample dataset containing similar distribution of normal transactions and Fraudulent Transactions

Number of Fraudulent Transactions --> 487

In [200]:
legit_sample = legit.sample(n=487)

Concatenating two DataFrames

In [201]:
new_dataset = pd.concat([legit_sample, fraud], axis=0)

In [202]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
237724,149366.0,2.105345,-0.131673,-1.656406,-0.066540,0.710193,0.151297,0.008382,-0.028435,0.327529,0.233118,-0.006885,0.784080,0.291653,0.372167,-0.548470,0.428442,-0.960924,0.001207,0.853151,-0.140766,-0.312979,-0.833859,0.228994,-0.348833,-0.162408,0.210748,-0.080313,-0.073748,1.98,0.0
65052,51411.0,-1.214709,1.414825,-0.285062,-1.776603,0.420260,0.452531,-0.155730,1.087302,-0.491037,-0.675458,-0.183740,1.061288,1.324637,0.507397,-0.111791,1.073082,-0.966376,0.095552,-0.175946,0.005911,-0.042716,-0.202039,-0.161071,-1.636244,0.050277,0.891793,0.058543,-0.005792,0.92,0.0
70497,53892.0,0.545471,-0.623443,-0.260729,1.456543,-0.176738,-0.423052,0.828627,-0.287433,-0.391535,-0.154925,-0.000041,0.806415,0.996161,0.379828,0.924527,-0.332951,-0.140916,-0.812978,-0.811010,0.580807,0.216769,-0.034845,-0.376397,0.126192,0.556201,-0.352783,-0.040073,0.075521,348.00,0.0
62546,50311.0,-4.385687,-2.135949,0.816318,2.213634,-2.158074,1.417995,0.879416,0.546345,1.168538,0.322079,-0.063568,0.386580,-0.963532,-0.525079,-1.048195,-0.157376,0.311671,0.884550,1.628739,-0.332296,-0.352909,0.565398,-0.149948,0.115918,0.852113,0.133031,0.950145,0.547172,552.35,0.0
271920,164811.0,-0.632350,-0.246712,0.103569,-1.604135,2.459825,4.008935,-0.371902,1.158833,0.210669,-0.811725,-0.471802,-0.082601,-0.187460,-0.122450,0.375965,0.193941,-0.633117,0.135689,-0.499769,0.159641,0.352086,0.867945,-0.041969,0.762038,-0.273250,0.613024,0.117492,0.160334,59.90,0.0


In [203]:
new_dataset.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
274382,165981.0,-5.766879,-8.402154,0.056543,6.950983,9.880564,-5.773192,-5.748879,0.721743,-1.076274,2.688670,-1.475145,-0.050468,0.113500,0.984344,0.363969,-0.674357,0.327464,0.091470,1.307941,2.493224,0.880395,-0.130436,2.241471,0.665346,-1.890041,-0.120803,0.073269,0.583799,0.00,1.0
274475,166028.0,-0.956390,2.361594,-3.171195,1.970759,0.474761,-1.902598,-0.055178,0.277831,-1.745854,-2.516628,0.874052,-2.513104,0.021575,-3.565119,0.461153,-2.015713,-1.731413,-0.465815,0.527620,0.190877,0.473211,0.719400,0.122458,-0.255650,-0.619259,-0.484280,0.683535,0.443299,39.90,1.0
275992,166831.0,-2.027135,-1.131890,-1.135194,1.086963,-0.010547,0.423797,3.790880,-1.155595,-0.063434,1.334414,1.032016,-0.722023,-1.533240,0.334119,0.297479,-0.429392,-0.824644,0.489668,0.873344,0.033804,-0.315105,0.575520,0.490842,0.756502,-0.142685,-0.602777,0.508712,-0.091646,634.30,1.0
276071,166883.0,2.091900,-0.757459,-1.192258,-0.755458,-0.620324,-0.322077,-1.082511,0.117200,-0.140927,0.249311,1.338318,-0.329759,-0.363591,-1.824839,-0.229679,1.842347,1.194212,0.037467,0.423099,0.037438,0.288253,0.831939,0.142007,0.592615,-0.196143,-0.136676,0.020182,-0.015470,19.95,1.0
276864,167338.0,-1.374424,2.793185,-4.346572,2.400731,-1.688433,0.111136,-0.922038,-2.149930,-2.027474,-4.390842,1.241958,-3.910522,-1.225636,-4.697997,0.676108,-2.077066,-2.683521,-0.525949,0.422342,0.650196,-0.870779,0.504849,0.137994,0.368275,0.103137,-0.414209,0.454982,0.096711,349.08,1.0


In [204]:
new_dataset['Class'].value_counts()

1.0    487
0.0    487
Name: Class, dtype: int64

In [205]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,94319.127310,-0.053513,0.063105,0.038286,-0.029982,-0.007686,-0.009296,0.105886,-0.051579,0.018751,-0.009901,0.050376,-0.006932,0.030816,0.097845,-0.084252,0.022915,-0.03101,-0.012196,0.034826,-0.037058,-0.007412,-0.014381,0.002614,0.002303,-0.030732,0.043055,-0.007270,-0.000727,87.269363
1.0,79834.240246,-4.816122,3.652183,-7.065000,4.576639,-3.179607,-1.398985,-5.612526,0.570034,-2.597563,-5.697347,3.820310,-6.291644,-0.103899,-6.993408,-0.090565,-4.164614,-6.70022,-2.263639,0.683637,0.372003,0.716151,0.014237,-0.041043,-0.104202,0.041249,0.047424,0.168312,0.075910,121.913326


## Data Preparation: feature engineering

In [206]:
X = new_dataset.drop(columns='Class', axis=1)
Y = new_dataset['Class']

In [207]:
print(X)

            Time        V1        V2  ...       V27       V28  Amount
237724  149366.0  2.105345 -0.131673  ... -0.080313 -0.073748    1.98
65052    51411.0 -1.214709  1.414825  ...  0.058543 -0.005792    0.92
70497    53892.0  0.545471 -0.623443  ... -0.040073  0.075521  348.00
62546    50311.0 -4.385687 -2.135949  ...  0.950145  0.547172  552.35
271920  164811.0 -0.632350 -0.246712  ...  0.117492  0.160334   59.90
...          ...       ...       ...  ...       ...       ...     ...
274382  165981.0 -5.766879 -8.402154  ...  0.073269  0.583799    0.00
274475  166028.0 -0.956390  2.361594  ...  0.683535  0.443299   39.90
275992  166831.0 -2.027135 -1.131890  ...  0.508712 -0.091646  634.30
276071  166883.0  2.091900 -0.757459  ...  0.020182 -0.015470   19.95
276864  167338.0 -1.374424  2.793185  ...  0.454982  0.096711  349.08

[974 rows x 30 columns]


In [208]:
print(Y)

237724    0.0
65052     0.0
70497     0.0
62546     0.0
271920    0.0
         ... 
274382    1.0
274475    1.0
275992    1.0
276071    1.0
276864    1.0
Name: Class, Length: 974, dtype: float64


In [209]:
# split into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [210]:
print(X.shape, X_train.shape, X_test.shape)

(974, 30) (779, 30) (195, 30)


## Model Training

Since our dependent variable is a Binary classification type so we can use Logistic Regression or k-Nearest Neighbors, Decision Trees, Support Vector Machine, Naive Bayes.

In [211]:
# Model with all features
from sklearn.linear_model import LogisticRegression

# training the Logistic Regression Model with Training Data
model=LogisticRegression()
model.fit(X_train, Y_train)

X_train_prediction = model.predict(X_train)

In [212]:
# Logistic Regression (LR)
# for Logistic Regression algorithm
# Training
from sklearn.linear_model import LogisticRegression
classifierLR = LogisticRegression()
classifierLR.fit(X_train,Y_train)
y_pred_train= classifierLR.predict(X_train)

# Evaluating the Algorithm on training data
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score  
print(confusion_matrix(Y_train, y_pred_train))  
print(classification_report(Y_train, y_pred_train))
print(accuracy_score(Y_train,y_pred_train))

[[377  12]
 [ 32 358]]
              precision    recall  f1-score   support

         0.0       0.92      0.97      0.94       389
         1.0       0.97      0.92      0.94       390

    accuracy                           0.94       779
   macro avg       0.94      0.94      0.94       779
weighted avg       0.94      0.94      0.94       779

0.9435173299101413


In [213]:
# Logistic Regression (LR)
# for Logistic Regression algorithm
# Training
from sklearn.linear_model import LogisticRegression
classifierLR = LogisticRegression()
classifierLR.fit(X_train,Y_train)
y_pred_test= classifierLR.predict(X_test)

# Evaluating the Algorithm on test data
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score  
print(confusion_matrix(Y_test, y_pred_test))  
print(classification_report(Y_test, y_pred_test))
print(accuracy_score(Y_test,y_pred_test))

[[92  6]
 [ 8 89]]
              precision    recall  f1-score   support

         0.0       0.92      0.94      0.93        98
         1.0       0.94      0.92      0.93        97

    accuracy                           0.93       195
   macro avg       0.93      0.93      0.93       195
weighted avg       0.93      0.93      0.93       195

0.9282051282051282


In [214]:
# Decision Tree
#for using Decision Tree Algoithm
# Training
from sklearn.tree import DecisionTreeClassifier
classifierDT = DecisionTreeClassifier()
classifierDT.fit(X_train,Y_train)
y_pred_train1= classifierDT.predict(X_train)

# Evaluating the Algorithm on training data
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score  
print(confusion_matrix(Y_train, y_pred_train1))  
print(classification_report(Y_train, y_pred_train1))
print(accuracy_score(Y_train,y_pred_train1))

[[389   0]
 [  0 390]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       389
         1.0       1.00      1.00      1.00       390

    accuracy                           1.00       779
   macro avg       1.00      1.00      1.00       779
weighted avg       1.00      1.00      1.00       779

1.0


In [215]:
# Decision Tree
#for using Decision Tree Algoithm
# Training
from sklearn.tree import DecisionTreeClassifier
classifierDT = DecisionTreeClassifier()
classifierDT.fit(X_train,Y_train)
y_pred_test1= classifierDT.predict(X_test)

# Evaluating the Algorithm on test data
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score  
print(confusion_matrix(Y_test, y_pred_test1))  
print(classification_report(Y_test, y_pred_test1))
print(accuracy_score(Y_test,y_pred_test1))

[[90  8]
 [10 87]]
              precision    recall  f1-score   support

         0.0       0.90      0.92      0.91        98
         1.0       0.92      0.90      0.91        97

    accuracy                           0.91       195
   macro avg       0.91      0.91      0.91       195
weighted avg       0.91      0.91      0.91       195

0.9076923076923077


In [216]:
# Support Vector Machine (SVM)
#for Support Vector Machine (SVM) Algorithm
# Training
from sklearn import svm
classifierSVM = svm.SVC()
classifierSVM.fit(X_train,Y_train)
y_pred_train2 = classifierSVM.predict(X_train)

# Evaluating the Algorithm on training data
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score  
print(confusion_matrix(Y_train, y_pred_train2))  
print(classification_report(Y_train, y_pred_train2))
print(accuracy_score(Y_train,y_pred_train2))

[[201 188]
 [179 211]]
              precision    recall  f1-score   support

         0.0       0.53      0.52      0.52       389
         1.0       0.53      0.54      0.53       390

    accuracy                           0.53       779
   macro avg       0.53      0.53      0.53       779
weighted avg       0.53      0.53      0.53       779

0.5288831835686778


In [217]:
# Support Vector Machine (SVM)
#for Support Vector Machine (SVM) Algorithm
# Training
from sklearn import svm
classifierSVM = svm.SVC()
classifierSVM.fit(X_train,Y_train)
y_pred_test2= classifierSVM.predict(X_test)

# Evaluating the Algorithm on test data
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score  
print(confusion_matrix(Y_test, y_pred_test2))  
print(classification_report(Y_test, y_pred_test2))
print(accuracy_score(Y_test,y_pred_test2))

[[52 46]
 [44 53]]
              precision    recall  f1-score   support

         0.0       0.54      0.53      0.54        98
         1.0       0.54      0.55      0.54        97

    accuracy                           0.54       195
   macro avg       0.54      0.54      0.54       195
weighted avg       0.54      0.54      0.54       195

0.5384615384615384


In [218]:
# K-Nearest Neighbors (KNN)
#for using K-Nearest Neighbors Algoithm
# Training
from sklearn.neighbors import KNeighborsClassifier
classifierKNN = KNeighborsClassifier(n_neighbors=5)
classifierKNN.fit(X_train,Y_train)
y_pred_train3 = classifierKNN.predict(X_train)

# Evaluating the Algorithm on training data
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score  
print(confusion_matrix(Y_train, y_pred_train3))  
print(classification_report(Y_train, y_pred_train3))
print(accuracy_score(Y_train,y_pred_train3))

[[302  87]
 [106 284]]
              precision    recall  f1-score   support

         0.0       0.74      0.78      0.76       389
         1.0       0.77      0.73      0.75       390

    accuracy                           0.75       779
   macro avg       0.75      0.75      0.75       779
weighted avg       0.75      0.75      0.75       779

0.7522464698331194


In [219]:
# K-Nearest Neighbors (KNN)
#for using K-Nearest Neighbors Algoithm
# Training
from sklearn.neighbors import KNeighborsClassifier
classifierKNN = KNeighborsClassifier(n_neighbors=5)
classifierKNN.fit(X_train,Y_train)
y_pred_test3= classifierKNN.predict(X_test)

# Evaluating the Algorithm on test data
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score  
print(confusion_matrix(Y_test, y_pred_test3))  
print(classification_report(Y_test, y_pred_test3))
print(accuracy_score(Y_test,y_pred_test3))

[[55 43]
 [39 58]]
              precision    recall  f1-score   support

         0.0       0.59      0.56      0.57        98
         1.0       0.57      0.60      0.59        97

    accuracy                           0.58       195
   macro avg       0.58      0.58      0.58       195
weighted avg       0.58      0.58      0.58       195

0.5794871794871795
